In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 548 kB 7.5 MB/s 
     |████████████████████████████████| 163 kB 51.3 MB/s 


# Library

In [ ]:
from sklearn.metrics import f1_score
from glob import glob
import pathlib
from pathlib import Path
from torchsummary import summary
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from pprint import pprint
import urllib.request
import csv
import numpy as np
#from einops import rearrange, reduce, repeat
from torch.cuda import amp
from tqdm import tqdm
#import wandb
import time
import copy
from collections import defaultdict
from sklearn.metrics import mean_squared_error
import joblib
import gc
import os
#from icecream import ic
from sklearn.model_selection import train_test_split
import gc
import cv2
import copy
import time
import random
from PIL import Image

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
from torchvision.transforms import ToTensor

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold

import timm

import json

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

# For colored terminal text
#from colorama import Fore, Back, Style
#c_ = Fore.CYAN
#sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


from sklearn.metrics import f1_score

# ENV


# SET SEED

In [ ]:
CONFIG = dict(
    nickname = 'convnext_small',
    seed=42,
    backbone='convnext_small',
    embedder=None,
    train_batch_size=8,
    valid_batch_size=16,
    img_size=224,
    num_epochs=50,
    early_stopping=False,
    early_stopping_step=5,
    learning_rate=1e-4,
    scheduler='CosineAnnealingLR',
    min_lr=1e-6,
    T_max=100,
    num_classes=25,
    weight_decay=1e-6,
    device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'),
    competition='lg',
    _wandb_kernel='deb'
)

# Augmentations

In [ ]:
# Augmentations 파트에서 resize,  augmentations, totensorV2 를 albumentation을 이용하여 객체 전달 후 getitem에서 전달해주기
# 사이즈가 정해져야 모델이 학습할 수 있음.

transform = A.Compose([
      A.Resize(224,224),
      A.Normalize(),
      ToTensorV2()
])


# Read the Data

In [ ]:
from IPython.utils.contexts import NoOpContext
class CustomImageDataset(Dataset) :
  def __init__(self, path, transform=None) :
    self.crops = []
    self.diseases = []
    self.risks = []
    self.label = {}
    labels = []
    
    self.json_list = sorted(glob(path+'crop_data/train/*/*.json'))
    self.jpg_list = sorted(glob(path+'crop_data/train/*/*.jpg'))
    
    for i in self.json_list :
      with open(i, 'r') as f :
        json_dict = json.load(f) # json_dict["annotations"]["crop"], json_dict["annotations"]["disease"], json_dict["annotations"]["risk"] 
        crop = json_dict["annotations"]["crop"]
        disease = json_dict["annotations"]["disease"]
        risk = json_dict["annotations"]["risk"]
      self.crops.append(crop)
      self.diseases.append(disease)
      self.risks.append(risk)
      labels.append(f"{crop}_{disease}_{risk}")
    np_label = np.array(labels)
    unique_label = np.unique(np_label)
    np.sort(unique_label)
    for x in range(len(unique_label.tolist())) :
      self.label[unique_label[x]] = x
    self.transform=transform
      
      
      
  def __len__(self) :
      return len(self.json_list)
    
  def __getitem__(self, idx):
      #tf_toTensor = ToTensor()

      img = cv2.imread(self.jpg_list[idx])
      #img_tensor = tf_toTensor(img)
      transformed = transform(image = img)
      img_tensor = transformed["image"]
      label_idx = f'{self.crops[idx]}_{self.diseases[idx]}_{self.risks[idx]}'
      return img_tensor, self.label[label_idx]
          

  

# Dataset

In [ ]:
 from google.colab.patches import cv2_imshow

In [ ]:
dataset = CustomImageDataset('/content/drive/MyDrive/',transform = transform)

In [ ]:
img = cv2.imread(dataset.jpg_list[100])
print(img[253][234][1])

125


In [ ]:
print(img.shape)
print(dataset[0])
print(len(dataset.label))

(512, 384, 3)
(tensor([[[-1.1589, -1.5014, -1.6898,  ..., -1.3987, -1.6042, -1.0219],
         [-1.1075, -1.1589, -1.6384,  ..., -0.5253, -1.3130, -1.0562],
         [-1.1247, -1.1247, -1.3987,  ..., -1.8439, -1.5528, -1.3644],
         ...,
         [ 1.1700,  1.2043,  1.1187,  ..., -1.8097, -1.8953, -1.8268],
         [ 1.1700,  1.1700,  1.1358,  ..., -1.6727, -1.8097, -1.7412],
         [ 1.2043,  1.1872,  1.0331,  ..., -1.6898, -1.7583, -1.7412]],

        [[ 0.0476, -0.0749, -0.1099,  ...,  1.4132,  0.9055,  1.1681],
         [ 0.1176,  0.1877, -0.1625,  ...,  1.4832,  0.6078,  0.7304],
         [ 0.0651,  0.1176, -0.0924,  ..., -0.0924,  0.2052,  0.2927],
         ...,
         [ 1.2381,  1.2381,  1.1856,  ..., -0.3725, -0.4426, -0.3550],
         [ 1.2031,  1.2031,  1.1506,  ..., -0.1800, -0.3200, -0.3025],
         [ 1.2556,  1.2206,  1.0630,  ..., -0.1450, -0.2675, -0.3200]],

        [[-0.6367, -0.8458, -0.8981,  ...,  0.6356,  0.1476,  0.4614],
         [-0.5670, -0.5147, -0

In [ ]:
#print(dataset[0]) # dataset 객체가 image tensor 와 해당하는 label을 반환
print(dataset.jpg_list[0])

/content/drive/MyDrive/crop_data/train/10027/10027.jpg


In [ ]:
# print(dataset.label['3_b7_1']) # key-value 잘 대응되는지 확인 완료

9


In [ ]:
loader = DataLoader(dataset, batch_size = 8, shuffle = True)

In [ ]:
print(dataset[0][0].shape)
print(dataset[0][1])

torch.Size([3, 224, 224])
9


#Model

In [ ]:
class Model(nn.Module) :
  def __init__(self, backbone, embedder, pretrained = True) :
    super(Model, self).__init__()
    self.backbone = timm.create_model(backbone, 
                                      pretrained = pretrained)
    self.backbone.reset_classifier(0)

  def forward(self, images) :
    
    output = self.backbone(images)
    return output



In [ ]:
X = torch.ones((CONFIG['train_batch_size'], 3, CONFIG['img_size'], CONFIG['img_size']))
net = Model(CONFIG['backbone'], CONFIG['embedder'])
output = net(X)


In [ ]:
print(output.shape)

torch.Size([8, 768])
